# Scraping & Cleaning BoardGameGeek.com Data

In this notebook, the goal is to scrape 10 pages of BGG.com (1000 games), clean the data, and pickle it for analysis in another notebook.


## Scraping BGG

In [1]:
import BGG
import pandas as pd

In [2]:
'''
BGG.py uses get() and get_stats() to browse through the given browse link page and returns
stats as a dictionary for each game in the page.

Here we search {pages} number of pages and that returns 100 games per page.
We then concatinate the results into one big dataframe "browse_df".
'''

pages = 10

browse_df_list = []
for idx in range(pages):
    browse_df_list.append(pd.DataFrame.from_dict(BGG.get(f'https://boardgamegeek.com/browse/boardgame/page/{idx+1}')))

browse_df = pd.concat(browse_df_list, ignore_index=True)

In [3]:
browse_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Name         1000 non-null   object 
 1   BGG_Rating   1000 non-null   float64
 2   AVG_Rating   1000 non-null   float64
 3   Voter_Count  1000 non-null   float64
 4   Min_Players  1000 non-null   int64  
 5   Max_Players  1000 non-null   int64  
 6   Min_Time     1000 non-null   int64  
 7   Max_Time     1000 non-null   int64  
 8   Min_Age      1000 non-null   int64  
 9   Difficulty   1000 non-null   float64
 10  Owners       1000 non-null   float64
 11  Total_Plays  1000 non-null   float64
dtypes: float64(6), int64(5), object(1)
memory usage: 93.9+ KB


In [4]:
browse_df.describe()

,BGG_Rating,AVG_Rating,Voter_Count,Min_Players,Max_Players,Min_Time,Max_Time,Min_Age,Difficulty,Owners,Total_Plays
count,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,6.955651,7.460800,9138.627000,1.90500,5.078000,64.040000,92.788000,11.438000,2.623085,13053.422000,33750.514000
std,0.370992,0.396179,11158.302695,0.70176,6.639729,53.204823,81.759315,2.295087,0.787178,15467.199896,56371.468511
min,6.490000,6.630000,754.000000,1.00000,1.000000,5.000000,10.000000,0.000000,1.018000,795.000000,760.000000
25%,6.659500,7.170000,2979.000000,2.00000,4.000000,30.000000,45.000000,10.000000,2.077375,4884.000000,7591.750000
50%,6.871000,7.410000,5192.500000,2.00000,4.000000,60.000000,75.000000,12.000000,2.587100,7687.500000,14945.000000
75%,7.167000,7.700000,10800.000000,2.00000,5.000000,75.000000,120.000000,13.000000,3.171825,14669.000000,34438.500000
max,8.572000,9.210000,95393.000000,8.00000,100.000000,480.000000,1000.000000,18.000000,4.726700,143004.000000,657153.000000


<h1 style="color:red;font-size = 20px;"><center><i>~~ Check Point ~~</i></center></h1>

## Outliers

1. min of Min_age shouldnt be 0 - set to mean (about 11years old)
2. max of Max_Players shouldnt be 100 players - values above 20 should be brought down to 20
3. max of Max_Time shouldnt be 1,000 minutes. Plus, it's always >= Min_Time - drop column


In [5]:
clean_df = browse_df.copy()

#### 1. Changing "min of Min_age"

In [6]:
avg_min_age = clean_df['Min_Age'].mean()
clean_df['Min_Age'].replace(0, avg_min_age, inplace=True)

print(clean_df['Min_Age'].min())   # should now be 4

4.0


#### 2. Changing "max of Max_Players"

In [7]:
# .where() is a little weird, because the condition should be for values that are to be
#    left alone. 
#    .where(<condition to leave value alone>, <value to change when condition false>)
clean_df['Max_Players'].where(clean_df['Max_Players'] <= 20, 20, inplace=True)

print(clean_df['Max_Players'].min(), clean_df['Max_Players'].max()) # should be (1, 20)

1 20


#### 3. Dropping "Max_Time"

In [8]:
clean_df.drop('Max_Time', axis=1, inplace=True)

clean_df.head()

,Name,BGG_Rating,AVG_Rating,Voter_Count,Min_Players,Max_Players,Min_Time,Min_Age,Difficulty,Owners,Total_Plays
0,Gloomhaven,8.572,8.82,36196.0,1,4,60,14.0,3.8355,58129.0,288034.0
1,Pandemic Legacy: Season 1,8.471,8.62,37811.0,2,4,60,13.0,2.8300,59680.0,214212.0
2,Brass: Birmingham,8.291,8.64,13746.0,2,4,60,14.0,3.9209,20305.0,33046.0
3,Terraforming Mars,8.280,8.43,56038.0,1,5,120,12.0,3.2357,73873.0,302058.0
4,Through the Ages: A New Story of Civilization,8.215,8.46,20639.0,2,4,120,14.0,4.4004,24324.0,69260.0


## Columns

1. Add a "Rank" column. The data is in order so should be {index+1}.
2. Add an "Avg_Plays" column. Calculation = ( Total_Plays / Owners ).
3. Remove 'Total_Plays' and 'Owners' columns


#### 1. Adding "Rank" Column

In [9]:
clean_df.reset_index(inplace=True)
clean_df.rename({'index': 'Rank'}, axis=1, inplace=True)
clean_df['Rank'] = clean_df['Rank'] + 1  # so it starts on 1

clean_df.head()

,Rank,Name,BGG_Rating,AVG_Rating,Voter_Count,Min_Players,Max_Players,Min_Time,Min_Age,Difficulty,Owners,Total_Plays
0,1,Gloomhaven,8.572,8.82,36196.0,1,4,60,14.0,3.8355,58129.0,288034.0
1,2,Pandemic Legacy: Season 1,8.471,8.62,37811.0,2,4,60,13.0,2.8300,59680.0,214212.0
2,3,Brass: Birmingham,8.291,8.64,13746.0,2,4,60,14.0,3.9209,20305.0,33046.0
3,4,Terraforming Mars,8.280,8.43,56038.0,1,5,120,12.0,3.2357,73873.0,302058.0
4,5,Through the Ages: A New Story of Civilization,8.215,8.46,20639.0,2,4,120,14.0,4.4004,24324.0,69260.0


#### 2 & 3. Adding "Avg_Plays" & Dropping 'Total_Plays' and 'Owners'

In [10]:
clean_df['Avg_Plays'] = round(clean_df['Total_Plays'] / clean_df['Owners'], 2)  # round to 2 decimal places
clean_df.drop(['Total_Plays', 'Owners'], axis=1, inplace=True)

clean_df.head()

,Rank,Name,BGG_Rating,AVG_Rating,Voter_Count,Min_Players,Max_Players,Min_Time,Min_Age,Difficulty,Avg_Plays
0,1,Gloomhaven,8.572,8.82,36196.0,1,4,60,14.0,3.8355,4.96
1,2,Pandemic Legacy: Season 1,8.471,8.62,37811.0,2,4,60,13.0,2.8300,3.59
2,3,Brass: Birmingham,8.291,8.64,13746.0,2,4,60,14.0,3.9209,1.63
3,4,Terraforming Mars,8.280,8.43,56038.0,1,5,120,12.0,3.2357,4.09
4,5,Through the Ages: A New Story of Civilization,8.215,8.46,20639.0,2,4,120,14.0,4.4004,2.85


<h1 style="color:orange;font-size = 20px;"><center><i>~~ Check Point ~~</i></center></h1>

## Categorical Data

1. "Min_Players" and "Max_Players" combined is nominal! (1-20) Find unique()'s Create Dummy Value columns. (Eg. "Plays1", "Plays2", etc.)
2. Remove "Min_Players" and "Max_Players" columns

In [11]:
dum_df = clean_df.copy()

In [12]:
# earlier we set the max Max_Players to 20 (and we know the min is 1)
for i in range(1, 21):
    mask = (i >= dum_df.loc[:, 'Min_Players']) & (i <= dum_df.loc[:, 'Max_Players'])
    dum_df[f'Plays_{i}'] = [1 if val == True else 0 for val in mask]

dum_df.drop(['Min_Players', 'Max_Players'], axis=1, inplace=True)

dum_df.columns

Index(['Rank', 'Name', 'BGG_Rating', 'AVG_Rating', 'Voter_Count', 'Min_Time',
       'Min_Age', 'Difficulty', 'Avg_Plays', 'Plays_1', 'Plays_2', 'Plays_3',
       'Plays_4', 'Plays_5', 'Plays_6', 'Plays_7', 'Plays_8', 'Plays_9',
       'Plays_10', 'Plays_11', 'Plays_12', 'Plays_13', 'Plays_14', 'Plays_15',
       'Plays_16', 'Plays_17', 'Plays_18', 'Plays_19', 'Plays_20'],
      dtype='object')

In [13]:
# Making sure that none of the new columns have all 0s or all 1s
# In other words, min of each column should be 0 and max of each should be 1
# Note: (Mean*100)% of games play that many players

dum_df[[f'Plays_{i+1}' for i in range(20)]].describe()

,Plays_1,Plays_2,Plays_3,Plays_4,Plays_5,Plays_6,Plays_7,Plays_8,Plays_9,Plays_10,Plays_11,Plays_12,Plays_13,Plays_14,Plays_15,Plays_16,Plays_17,Plays_18,Plays_19,Plays_20
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.00000,1000.00000,1000.00000,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.244000,0.881000,0.852000,0.859000,0.440000,0.194000,0.08300,0.063000,0.031000,0.030000,0.018000,0.018000,0.01200,0.01200,0.01200,0.01200,0.011000,0.011000,0.007000,0.007000
std,0.429708,0.323951,0.355278,0.348196,0.496635,0.395627,0.27602,0.243085,0.173404,0.170673,0.133018,0.133018,0.10894,0.10894,0.10894,0.10894,0.104355,0.104355,0.083414,0.083414
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.00000,1.00000,1.00000,1.000000,1.000000,1.000000,1.000000


**Small Note:** We don't drop a column here because the dummies are not mutually exclusive. eg. A board game could play 3 players AND 4 players. The inclusion of 1 doesnt imply the exclusion of another. 

**Bigger Note:** 11 and 12 have the same mean. This meaning that if a game plays 12 players, then it also plays 11. The same can be said for 13-16, 17&18, and 19&20. So we'll keep the last value of each interval 12, 16, 18, and 20, and drop 11, 13, 14, 15, 17, and 19.

In [14]:
to_drop = [11, 13, 14, 15, 17, 19]
dum_df.drop([f'Plays_{plct}' for plct in to_drop], axis=1, inplace=True)

dum_df.head()

,Rank,Name,BGG_Rating,AVG_Rating,Voter_Count,Min_Time,Min_Age,Difficulty,Avg_Plays,Plays_1,...,Plays_5,Plays_6,Plays_7,Plays_8,Plays_9,Plays_10,Plays_12,Plays_16,Plays_18,Plays_20
0,1,Gloomhaven,8.572,8.82,36196.0,60,14.0,3.8355,4.96,1,...,0,0,0,0,0,0,0,0,0,0
1,2,Pandemic Legacy: Season 1,8.471,8.62,37811.0,60,13.0,2.8300,3.59,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Brass: Birmingham,8.291,8.64,13746.0,60,14.0,3.9209,1.63,0,...,0,0,0,0,0,0,0,0,0,0
3,4,Terraforming Mars,8.280,8.43,56038.0,120,12.0,3.2357,4.09,1,...,1,0,0,0,0,0,0,0,0,0
4,5,Through the Ages: A New Story of Civilization,8.215,8.46,20639.0,120,14.0,4.4004,2.85,0,...,0,0,0,0,0,0,0,0,0,0


## Pickle Away

Data looks good! Let's pickle it and continue with EDA in another notebook.

In [15]:
import pickle

In [16]:
with open('bgg_df.pickle', 'wb') as to_write:
    pickle.dump(dum_df, to_write)

In [17]:
!ls

BGG.py                 Metis_Files            __pycache__
BGG_Scrape_Clean.ipynb Stats_Prework          bgg_df.pickle
Metis_2020_Submissions Untitled.ipynb         gitHub_blog


In [18]:
del browse_df, clean_df, dum_df